In [1]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Создаем spark сессию
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as sql_func

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', '16G')
    .config('spark.sql.analyzer.failAmbiguousSelfJoin', 'False')
    .master("local[*]")
    .getOrCreate()
)

In [2]:
DATA_DIR = 'D:/Datasets/ml-latest/'

In [6]:
# мы будем использовать данные, сохраненные в предыдущем ноутбуке
!ls {DATA_DIR}/tf_idf.parquet

_SUCCESS
part-00000-6183fc36-4fea-4420-95f2-7dd9b5817b37-c000.snappy.parquet


In [7]:
# создаем сессию Spark
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as sql_func

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', '16G')
    .config('spark.sql.analyzer.failAmbiguousSelfJoin', 'False')
    .master("local[*]")
    .getOrCreate()
)

In [9]:
# поскольку в parquet схема данных хранится внутри самого файла, читать их очень просто
tf_idf = spark.read.parquet(os.path.join(DATA_DIR, 'tf_idf.parquet')).cache()
tf_idf.show()

+-------+--------------------+--------------------+
|movieId|               title|              tf_idf|
+-------+--------------------+--------------------+
|    148|Awfully Big Adven...|(1024,[6,155,177,...|
|    463|Guilty as Sin (1993)|(1024,[219,263,58...|
|    471|Hudsucker Proxy, ...|(1024,[28,32,43,4...|
|    496|What Happened Was...|(1024,[43,187,219...|
|    833|High School High ...|(1024,[138,297,30...|
|   1088|Dirty Dancing (1987)|(1024,[16,26,39,6...|
|   1238|   Local Hero (1983)|(1024,[53,91,103,...|
|   1342|     Candyman (1992)|(1024,[22,89,115,...|
|   1580|Men in Black (a.k...|(1024,[33,35,36,4...|
|   1591|        Spawn (1997)|(1024,[2,32,33,14...|
|   1645|The Devil's Advoc...|(1024,[7,20,65,10...|
|   1829|  Chinese Box (1997)|(1024,[207,219,47...|
|   1959|Out of Africa (1985)|(1024,[20,32,62,6...|
|   2122|Children of the C...|(1024,[22,27,43,7...|
|   2142|American Tail: Fi...|(1024,[50,165,192...|
|   2366|    King Kong (1933)|(1024,[30,32,59,6...|
|   2659|It 

In [11]:
# считаем данные об оценках
ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, 'ratings.csv'),
        header=True,
        inferSchema=True
    )
    # только о миллионе записей, потому, что даже на таком объеме обсчет будет занимать до двух часов
    .limit(1000000)
    .select('movieId', 'userId', 'rating')
)

In [12]:
from sklearn.linear_model import ElasticNet
import numpy as np
from pyspark.sql.types import FloatType, ArrayType

def sklearn_lr(spark_x: list, spark_y: list) -> list:
    """
    spark_x: список pyspark.ml.linalg.SparseVector - фичи для регрессии
    spark_y: список занчений для целевой переменной регрессии
    return: список коэффициентов регрессии 
    """
    # переводим данные из формата Spark в формат sklearn
    numpy_x = np.array([vector.toArray() for vector in spark_x])
    numpy_y = np.array(spark_y).reshape(-1, 1)
    # применяем обычную модель из sklearn
    lr = ElasticNet().fit(numpy_x, numpy_y)
    # возвращаем в ответе плотный вектор коэффициентов регрессии
    return [lr.sparse_coef_.todense().tolist()[0], lr.intercept_.tolist()]

# определим spark UDF, которая обучит регрессию на своих аргументах
reg_udf = sql_func.udf(
    sklearn_lr,
    returnType=ArrayType(ArrayType(FloatType()))
)